# Imports

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px

# Params to Set

In [2]:
POSITION = "qb"
THRESHOLD = 1

# Load in Data

In [3]:
pickle_to_load = f"../../FantasyData/data-frames/df_{POSITION}_college_2000_2023.pkl"

In [4]:
df_pos = pd.read_pickle(pickle_to_load)

### For saving Data Later

In [5]:
save_path = f"../../interactive-2.0/{POSITION.upper()}/draft-capital/"

# Clean Up Data

In [6]:
df_clean = df_pos.copy().dropna(subset=["draft_ovr"])

In [7]:
df = df_clean[["draft_ovr", "player_name", "tier"]]

# Get Draft Capital Success Rate By Round

Defining pick ranges:

In [8]:
# bins = [0, 3, 5, 10, 15, 20, 32, float('inf')]  # This defines 'Top 3', 'Top 5', 'Top 10', and 'Others'
# labels = ['1-3', '4-5', '6-10', '11-15', '16-20', '21-32', "Others"]

In [9]:
# bins = [0, 10, 20, 32, 48, 64, 80, 96, float('inf')]  # This defines 'Top 3', 'Top 5', 'Top 10', and 'Others'
# labels = ['1-10', '11-20', '21-32', '33-48', '49-64', "65-80", "81-96", "Others"]

In [10]:
bins = [0, 5, 10, 32, float('inf')]  # This defines 'Top 3', 'Top 5', 'Top 10', and 'Others'
labels = ['1-5', '6-10', '11-32', "Others"]

Categorize each player into a pick range:

In [11]:
# Step 2: Categorize each player into a pick range
df['pick_range'] = pd.cut(df['draft_ovr'], bins=bins, labels=labels, right=True)

/tmp/ipykernel_22579/185501029.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pick_range'] = pd.cut(df['draft_ovr'], bins=bins, labels=labels, right=True)


In [12]:
pick_ranges = df[['player_name', 'pick_range']].drop_duplicates()

In [13]:
career_best_tier = df.groupby('player_name')['tier'].min().reset_index()

In [14]:
tier_counts = pd.merge(career_best_tier, pick_ranges, on='player_name')

In [15]:
total_players_per_range = tier_counts.groupby('pick_range')['player_name'].count().reset_index(name='total_players')

/tmp/ipykernel_22579/3307901133.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_players_per_range = tier_counts.groupby('pick_range')['player_name'].count().reset_index(name='total_players')


In [16]:
players_in_tier = tier_counts.groupby(['pick_range', 'tier']).size().reset_index(name='players_in_tier')

/tmp/ipykernel_22579/1144034013.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  players_in_tier = tier_counts.groupby(['pick_range', 'tier']).size().reset_index(name='players_in_tier')


In [17]:
# Merge to get the total players per pick range
merged_data = pd.merge(players_in_tier, total_players_per_range, on='pick_range')

# Calculate the odds as a percentage
merged_data['percentage'] = (merged_data['players_in_tier'] / merged_data['total_players']) * 100

In [18]:
# Filter out rows where 'Percentage' is 0
final_df = merged_data.copy().loc[merged_data['percentage'] != 0]

In [19]:
graph_df = final_df.query(f"pick_range != 'Others' and tier <= {THRESHOLD}").copy()

In [20]:
graph_df.reset_index(inplace=True)

In [21]:
graph_df.rename(columns={"percentage": "Hit Rate Percentage", 
                         "pick_range": 'Draft Position', 
                         "tier": "Best Fantasy Season",
                         'players_in_tier': f"Sample Size of {POSITION.upper()} Tier",
                         'total_players': 'All Players with this Draft Capital'
                        }, inplace=True)

# Highlight DF and Turn to HTML

In [25]:
# Define a function to highlight rows based on a condition 
def highlight_row(row): 
    # Mapping of pick ranges to background colors
#     color_map = {
#         '1-10': 'lightgreen',
#         '11-20': 'lightgrey',
#         '21-32': 'lightcoral',
#         '33-48': 'lightsalmon',
#         '49-64': 'lightseagreen',
#         '65-80': 'lightskyblue',
#         '81-96': 'lightsteelblue'
#     }

#     color_map = {
#         '1-3': 'lightgreen',     # High rank
#         '4-5': 'lightgrey',      # Slightly lower rank
#         '6-10': 'lightcoral',    # Middle of the pack
#         '11-15': 'lightsalmon',  # Getting towards the lower end
#         '16-20': 'lightseagreen',# Lower still
#         '21-32': 'lightskyblue', # Near the bottom
#         'Others': 'lightsteelblue'  # Everyone else
#     }

    color_map = {
        '1-5': 'lightgreen',     # High rank
        '6-10': 'lightcoral',    # Middle of the pack
        '11-32': 'lightskyblue', # Near the bottom
        'Others': 'lightsteelblue'  # Everyone else
    }
    
    # Get the background color for the current row's pick range
    # Default to white if the pick range is not in the map
    bg_color = color_map.get(row['Draft Position'], 'white')
    
    # Return a list of styles for each column in the row
    return [f'background-color: {bg_color}']*len(row)

In [26]:
# Apply the function to the DataFrame 
df_1 = graph_df.drop(columns="index").iloc[:12].style.apply(highlight_row, axis=1)

In [27]:
df_1

,Draft Position,Best Fantasy Season,Sample Size of QB Tier,All Players with this Draft Capital,Hit Rate Percentage
0,1-5,1,21,31,67.741935
1,6-10,1,5,10,50.000000
2,11-32,1,10,23,43.478261


In [28]:
with open(f'{save_path}/round1-better-samples.html', 'w') as file:
    file.write(df_1.to_html())

In [ ]:
df_2 = graph_df.drop(columns="index").iloc[18:].style.apply(highlight_row, axis=1)

In [ ]:
# with open(f'{save_path}/round2-3.html', 'w') as file:
#     file.write(df_2.to_html())

In [ ]:
df_2

In [29]:
df_summarized = graph_df.copy().drop(columns="index").groupby("Draft Position").agg({"Hit Rate Percentage": "sum"})

/tmp/ipykernel_22579/1321185484.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_summarized = graph_df.copy().drop(columns="index").groupby("Draft Position").agg({"Hit Rate Percentage": "sum"})


In [30]:
df_summarized["Bust Rate Percentage"] = 100 - df_summarized["Hit Rate Percentage"]

In [31]:
df_summarized.reset_index(inplace=True)

In [36]:
df_summarized = df_summarized.iloc[:3]

In [37]:
df_3 = df_summarized.style.apply(highlight_row, axis=1)

In [38]:
df_3

,Draft Position,Hit Rate Percentage,Bust Rate Percentage
0,1-5,67.741935,32.258065
1,6-10,50.000000,50.000000
2,11-32,43.478261,56.521739


In [39]:
df_summarized["Hit Rate Percentage"].mean()

53.74006545114539

In [40]:
with open(f'{save_path}/summarized-better-samples.html', 'w') as file:
    file.write(df_3.to_html())

# Plot

In [ ]:
fig = px.bar(graph_df, x="Best Fantasy Season", y="Hit Rate Percentage",
             color='Draft Position', barmode='group',
             hover_data=[f'Sample Size of {POSITION.upper()} Tier', 'All Players with this Draft Capital'],
             )

fig.write_html(f"{save_path}/hit-rate-v-tier.html")
fig.show()

In [ ]:
graph_df["Best Fantasy Season"] = graph_df["Best Fantasy Season"].astype(str)

In [ ]:
fig = px.bar(graph_df, x="Draft Position", y="Hit Rate Percentage",
             color='Best Fantasy Season', barmode='group',
             hover_data=[f'Sample Size of {POSITION.upper()} Tier', 'All Players with this Draft Capital'],
             )

fig.write_html(f"{save_path}/hit-rate-v-draft-pos.html")
fig.show()

# Further Exploration

In [ ]:
df_clean.query("draft_ovr <= 3")["player_name"].unique()

In [ ]:
df_clean.columns

In [ ]:
df_clean.query("draft_ovr >= 15 and draft_round == 1")["player_name"].unique()

In [43]:
15 / 33

0.45454545454545453